## Setup

In [ ]:
cd /u/sar-r2/fwilliam/Calif/UAVSAR/Stacks/Haywrd_05502_01-S1

## Download UAVSAR SLC files

In [ ]:
# !man wget
!wget -i UAVSAR_landslides/Haywrd_05502_01.txt --user=forrest --password=############